<a href="https://colab.research.google.com/github/Shaam93/Building-a-classifer-with-Pytorch/blob/master/DenseNet_Feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install torch==0.4.0 torchvision
!pip install torch torchvision
import torch
# this should print "torch version: 0.4.0"
print("torch version: ", torch.__version__)

# we need pillow version 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# then install the new one
!pip install Pillow==5.3.0
# import the new PIL version
import PIL
# this should print "pillow version: 5.3.0" 
# If it doesn't restart your runtime by going to Menu, Runtime, Restart Runtime
print("pillow version: ", PIL.PILLOW_VERSION)

torch version:  1.1.0
Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


pillow version:  5.3.0


In [0]:
import time
import json
import copy
import torch
import torchvision
from PIL import Image
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import lr_scheduler
from torchvision import transforms, models, datasets

In [0]:
#to mount your Google Drive, other option is to zip the folder of images you need then upload it to Google colab with the "Upload" button, and then unzipping the file here with code
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# TEST THE MODEL WITH VGG16 (PRETRAINED=FALSE)

# Test For CUDA

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
# Load the Data
from torchvision import datasets, transforms, models
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
#from tensorboard import SummaryWriter

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 10
# percentage of training set to use as validation
valid_size = 0.2
# Data directory (should be changed according to your path)
data_dir = 'gdrive/My Drive/Thesis/ModifiedSixClasses'

# Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
# choose the training and test datasets (change the path according to your data) you can put the path directly instead of: data_dir+'Train' for the train images
#or any other path to your images which you need to extract the features from
train_data = datasets.ImageFolder(data_dir + '/Train', transform=train_transforms)
#you can put the path directly instead of: data_dir+'Test' for the test images 
test_data = datasets.ImageFolder(data_dir + '/Test', transform=test_transforms)

#splitting the data into train, validation and test sets
# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
test_idx = [i for i in range(len(test_data))]
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    sampler=test_sampler,drop_last=False,num_workers=num_workers)

# specify the image classes
classes = ['1','2','3' ,'4' ,'5' ,'6']

print("Loaded the data successfully ...")

Loaded the data successfully ...


In [0]:
# Using & Viewing pretrained model (we do not change anything on the model here yet)
#example used model here is densenet121, you can change it to any other model 
print("Viewing the pretrained Model ...")
model = models.densenet121(pretrained=True)
print(model)

Viewing the pretrained Model ...


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:01<00:00, 24275100.74it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

**have a look now at the last output, look at the last layer, it is named "classifier", usually this is the layer that you use to get the number of the classes. Now we need to replaced its output with the number of the features we want in the feature vector. For exmaple in the following I wanted to have a feature vector of size 1024 for each image, feel free to change this as you wish based on the comments. **

In [0]:
import torch.nn as nn
import torch.nn.functional as F

print("Viewing the modified Network Architecture ...")

# define the layer you want to replace, you can also find this part online in some resources as the "identity" layer
class Classifier(nn.Module):
    def __init__(self):
       
        super(Classifier, self).__init__()
        ######important######
        #change this line according to how big you want your feature vector to be, here the feature vector will have a size of 1024
        #self.fc1= nn.Linear(first input depends on the last layer of the model before the "classifier" layer, or sometimes it is called "fc" layer, the output is the size of your feature vector)
        self.fc1 = nn.Linear(1024, 1024) 

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        return x


# create a complete CNN
model.classifier = Classifier()
#if the last layer in your model is called fc then use the following line
#model.fc =  Classifier()

Viewing the Network Architecture ...


'for param in model.parameters():\n    param.requires_grad = False'

In [0]:
#to test the output of your  model (the feature vector size)
x = torch.randn(1, 3, 224, 224)
output = model(x)

print(output.shape)

torch.Size([1, 1024])


In [0]:
# Get the featuer vectors of all the images in the test set, if you want to extract the features in the train set replace "test_loader" by "train_loader"
if train_on_gpu:
    model.cuda()
    
print("Getting the feature vectors of your images is in Progress ...")
FV= [] #to collect all the feature vectors

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    FV.append(output)
    #print(output.shape)
print("Testing the Trained Network is Finished")
print(FV)

In [0]:
# Get the featuer vectors of all the images in the test set, if you want to extract the features in the train set replace "test_loader" by "train_loader"
print("Getting the feature vectors of your images is in Progress ...")
# Test the Trained Network on CPU
labels= []
FV= []  #to collect all the feature vectors
counter= 0
model.cpu()
with torch.no_grad():
  for i, (data, targets) in enumerate(train_loader):
    output = model(data)
    FV.append(output.data.tolist())
    labels.append(targets.item())
    counter= i
    
print("Getting the feature vectors of your images is Finished!")
print("Number of Images which we extracted features from is: ", counter)

Testing the Trained Network is Finished
Number of Images 831


In [0]:
FV= np.array(FV) #convert the list into array to be able to reshape the feature vectors
#print(FV)
print(FV.shape)

[[[ 0.32370925  0.82736659 -0.27542758 ...  0.43905091 -0.34853292
    0.29555115]]

 [[ 0.34610209  0.83329141 -0.23297775 ...  0.4437449  -0.35099807
    0.26116562]]

 [[ 0.25344753  0.79795212 -0.25761664 ...  0.39657754 -0.36041403
    0.28936246]]

 ...

 [[ 0.31986853  0.8511312  -0.25927624 ...  0.46423244 -0.37248254
    0.29600784]]

 [[ 0.27990925  0.83055079 -0.24629395 ...  0.42959565 -0.35889852
    0.29659656]]

 [[ 0.32579982  0.80333358 -0.275161   ...  0.47051948 -0.32189947
    0.27233046]]]
(832, 1, 1024)


In [0]:
FV2=  np.empty((1024, 832)) #choose the shape you need, here I reshaped it this way because I need it to be like this (length of the Feature vector, number of images)
for i, val in enumerate(FV):
  val= val.reshape(1024, 1)
  np.concatenate((FV2, val), axis= 1)
 
#print(FV2)

[[ 0.32370925  0.82736659 -0.27542758 ... -1.10647118 -0.2693184
   0.9853121 ]
 [-0.02894734 -0.54835284  0.03568567 ... -0.29622668 -0.16395263
  -0.49134821]
 [-0.28973013  0.31401974 -0.22905207 ...  0.27917466 -0.01399475
  -0.18747541]
 ...
 [-0.10423304 -0.3947815   0.7056545  ... -0.44252345  0.34429666
   0.5190084 ]
 [-0.13648178  0.0267427   0.33430025 ... -0.14720182 -1.03976405
  -0.02464781]
 [-0.03570941  0.25726002 -0.19233489 ...  0.47051948 -0.32189947
   0.27233046]]


In [0]:
# not necessary for you but for me I needed it as a data frame to creat a correlation matrix
import pandas as pd
df= pd.DataFrame(FV2, columns= labels)

In [0]:
print(df)

             2         1         2  ...         4         5         6
0     0.323709  0.827367 -0.275428  ... -0.042195 -0.070360  0.007719
1    -0.028947 -0.548353  0.035686  ... -0.040962 -0.027828 -0.105555
2    -0.289730  0.314020 -0.229052  ... -0.010882  0.019849  0.008855
3     0.226208 -0.211631 -0.460487  ... -0.064621 -0.131572  0.061088
4    -0.042015 -0.488078  0.425623  ...  0.020321 -0.037796 -0.005671
5    -0.233459  0.039377 -0.371909  ... -0.004114  0.018708 -0.087440
6    -0.133548  0.480235 -0.922469  ... -0.011367 -0.020877 -0.111912
7     0.057377 -0.266844 -0.850585  ... -0.074769 -0.032866 -0.036245
8     0.222092  0.831864  0.302121  ... -0.013030 -0.046477 -0.145518
9    -0.996068 -0.904400 -0.459541  ...  0.005383 -0.018453 -0.064317
10    0.539975  0.031433  0.225427  ...  0.016733 -0.024616  0.055795
11   -0.176606 -0.472893 -0.319264  ... -0.030013 -0.038252 -0.005578
12    0.212455 -0.023769  0.460573  ...  0.020032 -0.012287  0.012153
13   -0.112499 -0.41